In [1]:
%load_ext autoreload
%autoreload 2

import os
print(os.getcwd())
import sys
sys.path.append(os.path.abspath(".."))

/root/code/Vivi-dsz/value_mapping/value_mapping/notebooks


In [26]:
import pandas as pd
from backend.preprocess.data_prep import *
from backend.preprocess.params import *
from backend.preprocess.alignment_score import get_alignment_summary
from backend.preprocess.brand_count_function import get_brand_counts
from backend.preprocess.review_count_function import get_review_counts
from data.raw.brands_about_us import brand_text


In [27]:
file_path = '/root/code/Vivi-dsz/value_mapping/value_mapping/data/raw/user_reviews_v01.csv'
reviews_df_full = pd.read_csv(file_path)
reviews_df_full.head()


,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,app
0,f97ba6d2-4d7c-4aab-ae45-a847584972cf,was using stocard everyday. without any notice...,1,0,25.29.239,2025-07-27 12:10:19,Hello and thank you for your review.\nThe loya...,2025-07-27 12:30:43,25.29.239,Klarna
1,e325f04e-0b7b-45ae-9013-a01a016baeab,Zero (0) of my cards transfered from Stocard. ...,1,0,25.29.239,2025-07-27 12:03:24,Thank you for the review. we understand that y...,2025-07-15 23:49:02,25.29.239,Klarna
2,f26beb63-d371-4759-8af7-cf4e0bf7421f,Nobody should use it. They ruined(acquired) th...,1,0,25.29.239,2025-07-27 06:28:26,Thank you for the review. We understand that y...,2025-07-27 06:49:25,25.29.239,Klarna
3,eed09a6f-c72a-4662-bd81-8d19d73a01d6,This Klarna app is garbage. When they switched...,1,0,25.29.239,2025-07-27 05:22:54,Thanks for the review. We are sorry to hear th...,2025-07-27 05:43:29,25.29.239,Klarna
4,1bcfeb1b-ca54-4883-9c12-34df63e6e8c4,"Absolutely horrendous, cannot transfer years o...",1,0,NaN,2025-07-27 01:22:31,Thank you for the review. We try to innovate o...,2025-07-27 01:41:27,NaN,Klarna


In [28]:
file_path = '/root/code/Vivi-dsz/value_mapping/value_mapping/data/preprocessed/topics_newkw_thousand.csv'


reviews_df =pd.read_csv(
    file_path,
    delimiter=';',
    encoding='ISO-8859-1',
    on_bad_lines='skip',  # Optional: skip malformed lines
    engine='python'  # Required when using `on_bad_lines` with older pandas
)

reviews_df

,Column1,reviewId,content,topic_openai,topic_category
0,0,00885280-2fda-46bf-9c24-142c644ab106,Used to be great. They changed my card and I h...,trust,trust_ethics
1,1,962be690-af75-4f76-94d4-76328aeeab83,"Very expensive, the plan costs more than brick...",affordable,quality_usability
2,2,a1730aab-1933-4b0c-8476-038583b54903,First I give 1 star because I had problem with...,service,user_centricity_support
3,3,7c32fad5-4eed-496d-bc51-a34905b8a22c,Great application It really work for me,functionality,quality_usability
4,4,aa8542cc-3282-4dd0-911c-dddac3d3a844,"Very good service, thank you.",service,user_centricity_support
...,...,...,...,...,...
995,995,0ec7e0a5-a70b-41e2-bcfd-95bd8c4142a7,Amazing app,experience,user_centricity_support
996,996,75d58234-4f4b-4650-a60b-9045211af444,gets stuck too often,reliability,quality_usability
997,997,64732b17-1a79-4244-8f42-a60d8575d16f,Al tratar de ingresar dinero la app crashea. A...,reliability,quality_usability
998,998,74941c3f-58c6-466c-9e3b-7d7ddbc369e0,Looking good,quality,quality_usability


In [30]:
# Merge the review_df with the reviews_df_full on reviewId to get apps

review_merged_df = pd.merge(
    reviews_df,
    reviews_df_full[['reviewId', 'app']],
    on='reviewId',
    how='left'
)


review_merged_df['app'] = review_merged_df['app'].replace({"TradeRepublic": "Trade Republic"})

In [31]:
review_merged_df.head()

,Column1,reviewId,content,topic_openai,topic_category,app
0,0,00885280-2fda-46bf-9c24-142c644ab106,Used to be great. They changed my card and I h...,trust,trust_ethics,Bunq
1,1,962be690-af75-4f76-94d4-76328aeeab83,"Very expensive, the plan costs more than brick...",affordable,quality_usability,Bunq
2,2,a1730aab-1933-4b0c-8476-038583b54903,First I give 1 star because I had problem with...,service,user_centricity_support,Bunq
3,3,7c32fad5-4eed-496d-bc51-a34905b8a22c,Great application It really work for me,functionality,quality_usability,Bunq
4,4,aa8542cc-3282-4dd0-911c-dddac3d3a844,"Very good service, thank you.",service,user_centricity_support,Bunq


In [32]:
keywords = [
    word
    for category, words in values_keywords.items()
    for word in [category] + words
]

len(keywords)

95

In [33]:
keywords = [
    word
    for words in values_keywords.values()
    for word in words
]

len(keywords)

87

In [34]:
brand_kw_df = get_brand_counts(brand_text, keywords)
review_kw_df = get_review_counts(review_merged_df, keywords)


In [35]:
print(brand_kw_df.shape)
print(review_kw_df.shape)


(5, 39)
(5, 33)


In [36]:
brand_kw_df.head()

,secure,growth,impact,smart,digital,accountability,transparency,responsibility,safety,control,...,social,ambition,empowerment,freedom,automation,ease,innovation,intuitive,reliability,flexibility
brand,,,,,,,,,,,,,,,,,,,,,
Klarna,5.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Revolut,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bunq,0.0,0.0,5.0,2.0,2.0,1.0,1.0,0.0,0.0,6.0,...,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
N26,10.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,3.0,...,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
Trade Republic,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [37]:
list(review_merged_df['app'].unique())

['Bunq', 'Klarna', 'N26', 'Revolut', 'Trade Republic']

In [38]:
review_kw_df.head()

,support,community,control,ai,trust,speed,reliability,digital,safety,smooth,...,innovation,functional,simplicity,ethics,smart,affordable,flexibility,integrity,impact,planning
brand,,,,,,,,,,,,,,,,,,,,,
Bunq,42.0,1.0,2.0,14.0,3.0,1.0,1.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Klarna,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N26,67.0,0.0,0.0,2.0,3.0,2.0,1.0,3.0,0.0,2.0,...,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0
Revolut,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
Trade Republic,54.0,0.0,0.0,4.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [39]:
brand_kw_df.head()

,secure,growth,impact,smart,digital,accountability,transparency,responsibility,safety,control,...,social,ambition,empowerment,freedom,automation,ease,innovation,intuitive,reliability,flexibility
brand,,,,,,,,,,,,,,,,,,,,,
Klarna,5.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Revolut,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bunq,0.0,0.0,5.0,2.0,2.0,1.0,1.0,0.0,0.0,6.0,...,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
N26,10.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,3.0,...,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
Trade Republic,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [40]:
for brand in list(review_merged_df['app'].unique()):
    summary = get_alignment_summary(brand, brand_kw_df, review_kw_df)
    print(f"{brand}: {summary}")

Bunq: {'brand': 'Bunq', 'Totall alignment_score': 0.452, 'Top values alignment score: ': 0.333, 'top_brand_values': ['user_centricity', 'growth', 'empowerment'], 'top_user_values': ['innovation', 'user_centricity', 'ethos'], 'misaligned_values': ['growth', 'empowerment', 'innovation', 'ethos'], 'recommendation': "Bunq emphasizes growth, empowerment in its messaging, but these values are not strongly perceived by users.Consider reinforcing them in UX, marketing, or customer service.Users perceive innovation, ethos as important, but these are not highlighted in the brand's communication.Consider reinforcing them in the brand communication."}
Klarna: {'brand': 'Klarna', 'Totall alignment_score': 0.156, 'Top values alignment score: ': 0.667, 'top_brand_values': ['user_centricity', 'quality'], 'top_user_values': ['user_centricity', 'simplicity', 'quality'], 'misaligned_values': ['simplicity'], 'recommendation': 'Klarna is well-aligned in its brand values and user perception.'}
N26: {'brand'

## Define Agent Tools (Function Wrappers)

In [14]:
# def analyze_brand_alignment(brand_name: str) -> str:
#     summary = get_alignment_summary(brand_name, brand_kw_df, review_kw_df, values_keywords)
#     if "error" in summary:
#         return summary["error"]

#     message = (
#         f"- Alignment Analysis for **{brand_name}**:\n"
#         f"- Alignment Score: {summary['alignment_score']:.2f}\n"
#         f"- Top Brand Values: {', '.join(summary['top_brand_values'])}\n"
#         f"- Top User Values: {', '.join(summary['top_user_values'])}\n"
#         f"- Misaligned Values: {', '.join(summary['misaligned_values']) or 'None'}\n"
#         f"- Recommendation: {summary['recommendation']}"
#     )
#     return message


## Set Up Agent Prompt

In [42]:
system_prompt = """
You are a senior brand strategy AI assistant helping professionals in marketing, advertising, and brand management.

You specialize in identifying misalignments between a brand’s messaging (from its official texts) and how users actually perceive that brand (from user reviews).

---

Available Tool:
You can call `get_alignment_summary(brand)` to:
- Compute a similarity or alignment score between brand and user values
- See the top brand-stated values vs top user-perceived values
- Identify gaps where the brand is over- or under-communicating
- Get a recommendation on what to reinforce or shift

Always call this function **before** making campaign suggestions.

---

Your Goals:
When asked about a brand’s strategy, perception, values, or ideas for campaigns:
1. Use `get_alignment_summary(...)` to gather insights
2. Highlight:
   - Misaligned values
   - Overused or under-recognized themes
   - Strategic risks or opportunities
3. Recommend campaign direction:
   - Messaging focus (which values to amplify or dial down)
   - Channels or formats (ads, content, UX, partnerships)
   - Taglines or positioning ideas

You are clear, confident, and helpful — like a brand strategist with insight from user behavior data.
"""


## Use GPT and Calling Tool Function

In [43]:
# from openai import OpenAI

# client = OpenAI(api_key= api_key)

# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": "What kind of campaign should Klarna run next?"}
# ]

# # Inject alignment analysis
# analysis = analyze_brand_alignment("Klarna")
# messages.append({"role": "assistant", "content": analysis})

# # Now ask GPT to reason on it
# messages.append({
#     "role": "user",
#     "content": "Based on that, what’s a creative campaign idea for Klarna?"
# })

# response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=messages,
#     temperature=0.7
# )

# print(response.choices[0].message.content)


In [ ]:
import openai
from openai import OpenAI
from backend.preprocess.alignment_score import get_alignment_summary
from backend.preprocess.params import values_keywords
import json

supported_brands = ["Klarna", "N26", "Revolut", "TradeRepublic", "Bunq"]

def extract_brand_names(text: str, brand_list=None):
    if brand_list is None:
        brand_list = supported_brands
    text_lower = text.lower()
    extracted_brand = [brand for brand in brand_list if brand.lower() in text_lower]
    return extracted_brand


def handle_query(question: str, brand_kw_df, review_kw_df, api_key):
    client = OpenAI(api_key=api_key)
    brands = extract_brand_names(question)
    print(brands)

    # If no brands found, return a message
    if not brands:
        return f"Sorry, I couldn't find any known brands in your question. Supported brands: {', '.join(supported_brands)}."

    elif len(brands) == 1:
        brand = brands[0]
        summary = get_alignment_summary(brand, brand_kw_df, review_kw_df)
        context = (
            f"You are analyzing the brand alignment for {brand}.\n"
            f"Use the JSON summary below to understand user perception vs brand values and generate strategy insights.\n\n"
            f"{json.dumps(summary, indent=2)}"
        )
        prompt = (
            f"You are a brand strategy consultant.\n\n"
            f"User asked: \"{question}\"\n\n"
            f"Use the summary below to answer with strategic recommendations:\n\n"
            f"{context}"
        )

    elif len(brands) == 2:
        b1, b2 = brands
        summary1 = get_alignment_summary(b1, brand_kw_df, review_kw_df)
        summary2 = get_alignment_summary(b2, brand_kw_df, review_kw_df)
        context = json.dumps({
            f"{b1}": summary1,
            f"{b2}": summary2
            }, indent=2)
        prompt = (
            f"You are a brand strategist comparing two brands.\n\n"
            f"User asked: \"{question}\"\n\n"
            f"Use the summaries below to compare their alignment with user perception."
            f"Highlight key differences and suggest which brand is better positioned:\n\n"
            f"{context}"
        )

    else:
        return "I can only compare up to two brands at a time. Please ask about one or two brands."

    # Send to OpenAI
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating response: {str(e)}"


In [45]:
# question = 'Is Klarna aligned with user perception?'
# response = handle_query(question, brand_kw_df, review_kw_df, api_key = os.getenv("OopenAI_API_KEY"))
# print(response)

question = 'How does Klarna win over N26?'
response = handle_query(question, brand_kw_df, review_kw_df, api_key = os.getenv("OopenAI_API_KEY"))
print(response)

# question = 'What kind of campaign should Klarna run next?'
# response = handle_query(question, brand_kw_df, review_kw_df)
# print(response)

['Klarna', 'N26']
Based on the alignment summaries for Klarna and N26, here are the key differences and strategic insights:

### Klarna:
- **Overall Alignment Score:** 0.156
- **Top Values Alignment Score:** 0.667
- **Top Brand Values:** User-centricity, Quality
- **Top User Values:** User-centricity, Simplicity, Quality
- **Misaligned Values:** Simplicity
- **Recommendation:** Klarna is well-aligned with user perceptions, particularly in user-centricity and quality. However, users also value simplicity, which is not as strongly communicated by Klarna.

### N26:
- **Overall Alignment Score:** 0.237
- **Top Values Alignment Score:** 0.333
- **Top Brand Values:** Innovation, User-centricity
- **Top User Values:** User-centricity, Simplicity, Ethos
- **Misaligned Values:** Innovation, Simplicity, Ethos
- **Recommendation:** N26 emphasizes innovation, but this is not strongly perceived by users. Users value simplicity and ethos, which are not prominently featured in N26's messaging.

### K